In [ ]:
################################################################################
# Molecular descriptors and fingerprints-based fitting using multiple algorithms
################################################################################
import pandas as pd
from sklearn.model_selection import train_test_split
from supervised.automl import AutoML
from sklearn.metrics import mean_squared_error, median_absolute_error, mean_absolute_error, roc_auc_score
from math import sqrt
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import multiprocessing
import os

class BestModel:
    def __init__(self, algorithm, metrics, model, predictions, y_test):
        self.algorithm = algorithm
        self.metrics = metrics
        self.model = model
        self.predictions = predictions
        self.y_test = y_test

def core():
    # Determine the number of CPU cores to use, with a maximum of 5
    return min(5, multiprocessing.cpu_count())

def calculate_metrics(y_true, y_pred):
    # Compute various evaluation metrics
    rmse = sqrt(mean_squared_error(y_true, y_pred))
    medae = median_absolute_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mre = mean_absolute_error(y_true, y_pred) / max(abs(y_true))
    medre = median_absolute_error(y_true, y_pred) / max(abs(y_true))
    try:
        roc = roc_auc_score(y_true, y_pred)
    except:
        roc = None  # ROC AUC only applies to classification tasks
    
    metrics = {
        "RMSE": rmse,
        "MedAE": medae,
        "MAE": mae,
        "MRE": mre,
        "MedRE": medre,
        "ROC": roc
    }
    return metrics

def train_and_evaluate(algorithm, X_train, y_train, X_test, y_test, results_path):
    try:
        # Initialize the AutoML framework with specified settings
        automl = AutoML(algorithms=[algorithm],
                        mode="Perform",
                        results_path=results_path,
                        eval_metric="rmse",
                        n_jobs=core(),
                        total_time_limit=3600,
                        model_time_limit=3600,
                        train_ensemble=True)
        automl.fit(X_train, y_train)

        # Predict and compute evaluation metrics
        pred_test = automl.predict(X_test)
        metrics = calculate_metrics(y_test, pred_test)
        return BestModel(algorithm, metrics, automl, pred_test, y_test)
    except Exception as e:
        print(f"Error occurred with {algorithm} for {results_path}: {e}")
        return None

def process_data(data_file):
    # Read the dataset
    df = pd.read_csv(data_file, low_memory=False)

    # Convert all feature columns to numeric, setting invalid values to NaN
    X = df.loc[:, df.columns != 'RI'].apply(pd.to_numeric, errors='coerce')
    y = df['RI'].apply(pd.to_numeric, errors='coerce')

    # No imputation or missing value handling is applied
    return X, y

def save_metrics_to_csv(best_model_info, data_folder, feature_type):
    # Save evaluation metrics to a CSV file
    metrics_df = pd.DataFrame([best_model_info.metrics])
    metrics_df.insert(0, 'Algorithm', best_model_info.algorithm)
    metrics_df.to_csv(os.path.join(data_folder, f"{feature_type}_model_metrics.csv"), index=False)

def save_predictions_to_csv(best_model_info, data_folder, feature_type):
    # Save prediction results to a CSV file
    predictions_df = pd.DataFrame({
        'Actual RI': best_model_info.y_test,
        'Predicted RI': best_model_info.predictions
    })
    predictions_df.to_csv(os.path.join(data_folder, f"{feature_type}_predictions.csv"), index=False)

def main():
    # Specify the data folder
    data_folder = "data\Modol"
    descriptors_file = os.path.join(data_folder, "DP_Training.csv")
    fingerprints_file = os.path.join(data_folder, "FP_Training.csv")
    model_save_path = os.path.join(data_folder, "best_model")

    # Create separate storage folders for each dataset type
    descriptors_folder = os.path.join(data_folder, "DP_model")
    fingerprints_folder = os.path.join(data_folder, "FP_model")
    os.makedirs(descriptors_folder, exist_ok=True)
    os.makedirs(fingerprints_folder, exist_ok=True)

    # Process datasets
    X_descriptors, y_descriptors = process_data(descriptors_file)
    X_fingerprints, y_fingerprints = process_data(fingerprints_file)

    # Print initial dataset statistics
    print(f"Descriptors dataset size: {X_descriptors.shape[0]} samples")
    print(f"Fingerprints dataset size: {X_fingerprints.shape[0]} samples")

    # Split datasets into training and testing sets
    X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_descriptors, y_descriptors, test_size=0.20, random_state=42)
    X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(X_fingerprints, y_fingerprints, test_size=0.20, random_state=42)

    # Print sizes of training and testing sets
    print(f"Descriptors training set size: {X_train_d.shape[0]} samples")
    print(f"Descriptors test set size: {X_test_d.shape[0]} samples")
    print(f"Fingerprints training set size: {X_train_f.shape[0]} samples")
    print(f"Fingerprints test set size: {X_test_f.shape[0]} samples")

    # Define algorithms to be tested
    all_algorithms = ["Xgboost", "CatBoost", "Decision Tree", "Extra Trees", "LightGBM"]

    # Train models in parallel
    results_d = joblib.Parallel(n_jobs=core())(
        joblib.delayed(train_and_evaluate)(algorithm, X_train_d, y_train_d, X_test_d, y_test_d, os.path.join(descriptors_folder, f"AutoML_{algorithm}"))
        for algorithm in all_algorithms
    )

    results_f = joblib.Parallel(n_jobs=core())(
        joblib.delayed(train_and_evaluate)(algorithm, X_train_f, y_train_f, X_test_f, y_test_f, os.path.join(fingerprints_folder, f"AutoML_{algorithm}"))
        for algorithm in all_algorithms
    )

    best_model_info_d = None
    best_rmse_d = float('inf')

    best_model_info_f = None
    best_rmse_f = float('inf')

    # Identify the best model for descriptors
    for result in results_d:
        if result and result.metrics["RMSE"] < best_rmse_d:
            best_rmse_d = result.metrics["RMSE"]
            best_model_info_d = result

    # Identify the best model for fingerprints
    for result in results_f:
        if result and result.metrics["RMSE"] < best_rmse_f:
            best_rmse_f = result.metrics["RMSE"]
            best_model_info_f = result

    # Print the best algorithm and model information
    if best_model_info_d:
        print(f"\nBest Algorithm for Descriptors: {best_model_info_d.algorithm}")
        for metric, value in best_model_info_d.metrics.items():
            print(f"Best Test {metric} for Descriptors: {value}")

        # Visualize prediction results
        plt.figure(figsize=(10, 6))
        sns.scatterplot(x=best_model_info_d.y_test, y=best_model_info_d.predictions, alpha=0.7)
        plt.plot([min(best_model_info_d.y_test), max(best_model_info_d.y_test)], [min(best_model_info_d.y_test), max(best_model_info_d.y_test)], linestyle='--', color='red')
        plt.title("Actual vs. Predicted RI (Retention Index) - Descriptors")
        plt.xlabel("Actual RI")
        plt.ylabel("Predicted RI")
        plt.savefig(os.path.join(descriptors_folder, "prediction_vs_actual_descriptors.png"))
        plt.close()

        # Save the best model and feature names
        joblib.dump(best_model_info_d.model, f"{model_save_path}_descriptors.joblib")
        joblib.dump(X_descriptors.columns, os.path.join(descriptors_folder, "feature_names_descriptors.joblib"))
        
        # Save evaluation metrics and predictions
        save_metrics_to_csv(best_model_info_d, descriptors_folder, "descriptors")
        save_predictions_to_csv(best_model_info_d, descriptors_folder, "descriptors")

    if best_model_info_f:
        print(f"\nBest Algorithm for Fingerprints: {best_model_info_f.algorithm}")
        for metric, value in best_model_info_f.metrics.items():
            print(f"Best Test {metric} for Fingerprints: {value}")

        # Visualize prediction results
        plt.figure(figsize=(10, 6))
        sns.scatterplot(x=best_model_info_f.y_test, y=best_model_info_f.predictions, alpha=0.7)
        plt.plot([min(best_model_info_f.y_test), max(best_model_info_f.y_test)], [min(best_model_info_f.y_test), max(best_model_info_f.y_test)], linestyle='--', color='red')
        plt.title("Actual vs. Predicted RI (Retention Index) - Fingerprints")
        plt.xlabel("Actual RI")
        plt.ylabel("Predicted RI")
        plt.savefig(os.path.join(fingerprints_folder, "prediction_vs_actual_fingerprints.png"))
        plt.close()

        # Save the best model and feature names
        joblib.dump(best_model_info_f.model, f"{model_save_path}_fingerprints.joblib")
        joblib.dump(X_fingerprints.columns, os.path.join(fingerprints_folder, "feature_names_fingerprints.joblib"))
        
        # Save evaluation metrics and predictions
        save_metrics_to_csv(best_model_info_f, fingerprints_folder, "fingerprints")
        save_predictions_to_csv(best_model_info_f, fingerprints_folder, "fingerprints")

    # Compare the best models from both features and select the overall best model
    if best_model_info_d and best_model_info_f:
        if best_model_info_d.metrics["RMSE"] < best_model_info_f.metrics["RMSE"]:
            overall_best_model = best_model_info_d
        else:
            overall_best_model = best_model_info_f
    elif best_model_info_d:
        overall_best_model = best_model_info_d
    elif best_model_info_f:
        overall_best_model = best_model_info_f
    else:
        overall_best_model = None

    if overall_best_model:
        print(f"\nOverall Best Algorithm: {overall_best_model.algorithm}")
        for metric, value in overall_best_model.metrics.items():
            print(f"Overall Best Test {metric}: {value}")

        # Save the overall best model and feature names
        joblib.dump(overall_best_model.model, f"{model_save_path}_overall_best.joblib")
        feature_names_path = os.path.join(data_folder, "feature_names_descriptors.joblib") if overall_best_model == best_model_info_d else os.path.join(data_folder, "feature_names_fingerprints.joblib")
        joblib.dump(X_descriptors.columns if overall_best_model == best_model_info_d else X_fingerprints.columns, feature_names_path)
        
        # Save predictions for the overall best model
        save_predictions_to_csv(overall_best_model, data_folder, "overall_best")

if __name__ == "__main__":
    main()


In [ ]:
###############################
# Evaluation of Best Model #
###############################
import os
import joblib
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.metrics import mean_squared_error, median_absolute_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import mahalanobis
from math import sqrt
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm

class MLWorkflow:
    def __init__(self, data_folder, descriptors_file, model_save_path, feature_names_path, evaluation_folder, target_column='RI'):
        self.data_folder = data_folder
        self.descriptors_file = os.path.join(data_folder, descriptors_file)
        self.model_save_path = os.path.join(data_folder, model_save_path)
        self.feature_names_path = os.path.join(data_folder, feature_names_path)
        self.evaluation_folder = os.path.join(data_folder, evaluation_folder)
        self.target_column = target_column
        self.model = None
        self.feature_names = None

        if not os.path.exists(self.evaluation_folder):
            os.makedirs(self.evaluation_folder)

    def load_model(self):
        try:
            self.model = joblib.load(self.model_save_path)
            print("Model loaded successfully.")
        except Exception as e:
            print(f"Error loading model: {e}")

    def load_feature_names(self):
        try:
            self.feature_names = joblib.load(self.feature_names_path)
            print("Feature names loaded successfully.")
        except Exception as e:
            print(f"Error loading feature names: {e}")

    def process_data(self, file_path):
        df = pd.read_csv(file_path, low_memory=False)

        # Ensure the dataset has an ID column (add it if not present)
        if 'ID' not in df.columns:
            df['ID'] = np.arange(len(df))
            print("ID column added to data.")

        # Convert non-numeric data to NaN
        df = df.apply(pd.to_numeric, errors='coerce')

        # Replace all NaN values with 0
        df = df.fillna(0)

        X = df.drop(columns=[self.target_column], errors='ignore')
        y = df[self.target_column]
        return X, y

    def stratified_split(self, X, y, test_size=0.2, bins=10):
        # Create stratified bins based on the target variable
        y_binned = pd.cut(y, bins=bins, labels=False)

        stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=42)
        train_index, test_index = next(stratified_split.split(X, y_binned))
        
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        return X_train, X_test, y_train, y_test

    def calculate_metrics(self, y_true, y_pred):
        # Calculate various evaluation metrics
        rmse = sqrt(mean_squared_error(y_true, y_pred))
        medae = median_absolute_error(y_true, y_pred)
        mae = mean_absolute_error(y_true, y_pred)
        mre = mean_absolute_error(y_true, y_pred) / max(abs(y_true))
        medre = median_absolute_error(y_true, y_pred) / max(abs(y_true))

        metrics = {
            "RMSE": rmse,
            "MedAE": medae,
            "MAE": mae,
            "MRE": mre,
            "MedRE": medre
        }

        return metrics

    def permutation_test(self, model, X_test, y_test, n_permutations=50):
        try:
            original_score = model.score(X_test, y_test)
        except Exception as e:
            print(f"Error in original score calculation: {e}")
            original_score = None

        permuted_scores = []
        for _ in tqdm(range(n_permutations), desc="Permutation Test"):
            y_test_permuted = np.random.permutation(y_test)
            try:
                permuted_score = model.score(X_test, y_test_permuted)
                permuted_scores.append(permuted_score)
            except Exception as e:
                print(f"Error in permutation score calculation: {e}")
                permuted_scores.append(None)

        return original_score, permuted_scores

    def train_and_validate_model(self, X, y):
        # Remove the ID column for training
        X = X.drop(columns=['ID'])

        # Split the data into training and validation sets
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

        # Assuming the use of a Random Forest model for training
        from sklearn.ensemble import RandomForestRegressor
        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

        # Validate the model
        y_val_pred = model.predict(X_val)
        val_metrics = self.calculate_metrics(y_val, y_val_pred)
        print(f"Validation Metrics: {val_metrics}")

        # Save the retrained model
        joblib.dump(model, self.model_save_path)
        print("Model retrained and saved successfully.")
        return model

    def plot_histogram(self, data, original_score, xlabel, ylabel, title, save_path, color='blue'):
        # Plot a histogram with the original score highlighted
        plt.figure(figsize=(10, 6))
        sns.histplot(data, kde=True, bins=30, color=color, alpha=0.7)
        plt.axvline(original_score, color='red', linestyle='--', label='Original Score')
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.title(title)
        plt.legend()
        plt.savefig(save_path)
        plt.close()

    def plot_williams(self, X_train, X_test, y_train, y_test, y_pred, save_path):
        # Generate a Williams plot (leverage vs standardized residuals)
        if 'ID' in X_train.columns:
            X_train = X_train.drop(columns=['ID'])
        if 'ID' in X_test.columns:
            X_test = X_test.drop(columns=['ID'])

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        leverage = np.sum(X_test_scaled * np.dot(X_test_scaled, np.linalg.pinv(np.dot(X_train_scaled.T, X_train_scaled))), axis=1)

        residuals = y_test - y_pred
        standardized_residuals = residuals / np.std(residuals)

        plt.figure(figsize=(10, 6))
        plt.scatter(leverage, standardized_residuals, alpha=0.6)
        plt.axhline(y=0, color='black', linestyle='--')
        plt.axhline(y=3, color='red', linestyle='--')
        plt.axhline(y=-3, color='red', linestyle='--')
        plt.axvline(x=3 * np.mean(leverage), color='blue', linestyle='--')
        plt.xlabel('Leverage')
        plt.ylabel('Standardized Residuals')
        plt.title('Williams Plot')
        plt.savefig(save_path)
        plt.close()

    def applicability_domain(self, X_train, X_test, epsilon=1e-5):
        # Evaluate the applicability domain using Mahalanobis distance
        if 'ID' in X_train.columns:
            X_train = X_train.drop(columns=['ID'])
        if 'ID' in X_test.columns:
            X_test = X_test.drop(columns=['ID'])

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Compute covariance matrix with regularization
        cov_matrix = np.cov(X_train_scaled, rowvar=False) + epsilon * np.eye(X_train_scaled.shape[1])
        
        try:
            inv_cov_matrix = np.linalg.inv(cov_matrix)
        except np.linalg.LinAlgError:
            print("Error: Covariance matrix is singular, applying further regularization.")
            inv_cov_matrix = np.linalg.pinv(cov_matrix)  # Use pseudo-inverse

        mahalanobis_distances = []
        for x in tqdm(X_test_scaled, desc="Applicability Domain"):
            dist = mahalanobis(x, np.mean(X_train_scaled, axis=0), inv_cov_matrix)
            mahalanobis_distances.append(dist)

        return np.array(mahalanobis_distances)

    def plot_applicability_domain(self, mahalanobis_distances, residuals, save_path):
        # Plot applicability domain analysis
        plt.figure(figsize=(10, 6))
        plt.scatter(mahalanobis_distances, residuals, alpha=0.6)
        plt.axhline(y=0, color='black', linestyle='--')
        plt.axvline(x=np.mean(mahalanobis_distances) + 2 * np.std(mahalanobis_distances), color='red', linestyle='--', label='AD Boundary')
        plt.xlabel('Mahalanobis Distance')
        plt.ylabel('Residuals')
        plt.title('Applicability Domain Analysis')
        plt.legend()
        plt.savefig(save_path)
        plt.close()

    def main(self):
        self.load_model()
        self.load_feature_names()
        
        # Load the dataset
        X, y = self.process_data(self.descriptors_file)
        
        # Verify if feature names exist in the dataset
        valid_feature_names = [feature for feature in self.feature_names if feature in X.columns]
        
        if len(valid_feature_names) < len(self.feature_names):
            missing_features = set(self.feature_names) - set(valid_feature_names)
            print(f"Warning: The following features are missing from the dataset and will be ignored: {missing_features}")
        
        # Use only valid features present in the data
        X_features = X[valid_feature_names]  
        
        # Perform stratified sampling
        X_train, X_test, y_train, y_test = self.stratified_split(X_features, y, test_size=0.20, bins=10)

        # Retrain and validate the model
        self.model = self.train_and_validate_model(X_train, y_train)

        # Drop the ID column from the test set before prediction
        X_test = X_test.drop(columns=['ID'])

        # Generate predictions and evaluate
        y_pred = self.model.predict(X_test)
        metrics = self.calculate_metrics(y_test, y_pred)
        print(f"Model Metrics: {metrics}")

        # Perform permutation testing
        original_score, permuted_scores = self.permutation_test(self.model, X_test, y_test, n_permutations=50)
        print(f"Original score: {original_score}")
        print(f"Permutation test scores: {permuted_scores}")
        self.plot_histogram(permuted_scores, original_score, 'Permuted Scores', 'Frequency',
                            'Permutation Test Scores', os.path.join(self.evaluation_folder, 'permutation_test.png'), color='blue')

        # Save Permutation Test results to CSV
        permuted_scores_df = pd.DataFrame(permuted_scores, columns=["Permuted Scores"])
        permuted_scores_df.to_csv(os.path.join(self.evaluation_folder, 'permutation_test_scores.csv'), index=False)

        # Generate Williams plot
        self.plot_williams(X_train, X_test, y_train, y_test, y_pred,
                           os.path.join(self.evaluation_folder, 'williams_plot.png'))

        # Evaluate applicability domain
        mahalanobis_distances = self.applicability_domain(X_train, X_test)
        residuals = y_test - y_pred
        self.plot_applicability_domain(mahalanobis_distances, residuals,
                                       os.path.join(self.evaluation_folder, 'applicability_domain.png'))

if __name__ == "__main__":
    workflow = MLWorkflow(data_folder="data\\Model",
                          descriptors_file="DP_Training.csv",
                          model_save_path="best_model_overall_best.joblib",
                          feature_names_path="feature_names_descriptors.joblib",
                          evaluation_folder="Evaluation",
                          target_column='RI')
    workflow.main()


In [4]:
##################################
# Analysis of feature importance #
##################################
import os
import joblib
import pandas as pd
import numpy as np
import shap
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.metrics import mean_squared_error, median_absolute_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import mahalanobis
from math import sqrt
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor


class MLWorkflow:
    def __init__(self, data_folder, descriptors_file, model_save_path, feature_names_path, evaluation_folder, target_column='RI'):
        self.data_folder = data_folder
        self.descriptors_file = os.path.join(data_folder, descriptors_file)
        self.model_save_path = os.path.join(data_folder, model_save_path)
        self.feature_names_path = os.path.join(data_folder, feature_names_path)
        self.evaluation_folder = os.path.join(data_folder, evaluation_folder)
        self.target_column = target_column
        self.model = None
        self.feature_names = None

        if not os.path.exists(self.evaluation_folder):
            os.makedirs(self.evaluation_folder)

    def load_model(self):
        try:
            self.model = joblib.load(self.model_save_path)
            print("Model loaded successfully.")
        except Exception as e:
            print(f"Error loading model: {e}")

    def load_feature_names(self):
        try:
            self.feature_names = joblib.load(self.feature_names_path)
            print("Feature names loaded successfully.")
        except Exception as e:
            print(f"Error loading feature names: {e}")

    def process_data(self, file_path):
        df = pd.read_csv(file_path, low_memory=False)

        if 'ID' not in df.columns:
            df['ID'] = np.arange(len(df))
            print("ID column added to data.")

        df = df.apply(pd.to_numeric, errors='coerce')
        df = df.fillna(0)

        X = df.drop(columns=[self.target_column], errors='ignore')
        y = df[self.target_column]
        return X, y

    def stratified_split(self, X, y, test_size=0.2, bins=10):
        y_binned = pd.cut(y, bins=bins, labels=False)
        stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=42)
        train_index, test_index = next(stratified_split.split(X, y_binned))
        return X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]

    def calculate_metrics(self, y_true, y_pred):
        rmse = sqrt(mean_squared_error(y_true, y_pred))
        medae = median_absolute_error(y_true, y_pred)
        mae = mean_absolute_error(y_true, y_pred)
        mre = mae / max(abs(y_true))
        medre = medae / max(abs(y_true))
        return {"RMSE": rmse, "MedAE": medae, "MAE": mae, "MRE": mre, "MedRE": medre}

    def train_and_validate_model(self, X, y):
        X = X.drop(columns=['ID'])
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

        y_val_pred = model.predict(X_val)
        val_metrics = self.calculate_metrics(y_val, y_val_pred)
        print(f"Validation Metrics: {val_metrics}")

        joblib.dump(model, self.model_save_path)
        print("Model retrained and saved successfully.")
        return model

    def calculate_shap_values(self, model, X):
        try:
            explainer = shap.Explainer(model, X)
            shap_values = explainer(X)
            print("SHAP values calculated successfully.")
            return explainer, shap_values
        except Exception as e:
            print(f"Error calculating SHAP values: {e}")
            return None, None

    def plot_global_feature_importance(self, shap_values, X, save_path):
        """
        Plot and save the global feature importance as a bar chart with adjusted display parameters.
        """
        plt.figure(figsize=(12, 8))  # Increased figure size for better clarity
        shap.summary_plot(shap_values, X, show=False, plot_size=(10, 6))  # Adjust plot size
        plt.title("Global Feature Importance (SHAP Summary Plot)", fontsize=16)  # Increased font size
        plt.savefig(save_path, bbox_inches='tight', dpi=300)  # Save with high resolution
        plt.close()
        print(f"Global feature importance plot saved to {save_path}")

    def plot_local_feature_importance(self, shap_values, X, instance_idx, save_path):
        """
        Plot and save the local feature importance (waterfall plot) for a single instance.
        """
        plt.figure(figsize=(12, 8))  # Increased figure size
        shap.waterfall_plot(
            shap_values[instance_idx],
            show=False,
            max_display=10  # Limit to top 10 features for better clarity
        )
        plt.title(f"Local Feature Importance for Instance {instance_idx}", fontsize=16)  # Increased font size
        plt.xlabel("Impact on Model Output", fontsize=14)  # Custom X-axis label
        plt.ylabel("Features", fontsize=14)  # Custom Y-axis label
        plt.savefig(save_path, bbox_inches='tight', dpi=300)  # Save with high resolution
        plt.close()
        print(f"Local feature importance plot for instance {instance_idx} saved to {save_path}")

    def main(self):
        self.load_model()
        self.load_feature_names()

        X, y = self.process_data(self.descriptors_file)
        valid_feature_names = [f for f in self.feature_names if f in X.columns]
        if len(valid_feature_names) < len(self.feature_names):
            missing_features = set(self.feature_names) - set(valid_feature_names)
            print(f"Warning: Missing features ignored: {missing_features}")
        X_features = X[valid_feature_names]

        X_train, X_test, y_train, y_test = self.stratified_split(X_features, y, test_size=0.20, bins=10)

        self.model = self.train_and_validate_model(X_train, y_train)

        X_test = X_test.drop(columns=['ID'])
        y_pred = self.model.predict(X_test)
        metrics = self.calculate_metrics(y_test, y_pred)
        print(f"Model Metrics: {metrics}")

        explainer, shap_values = self.calculate_shap_values(self.model, X_test)
        if shap_values is not None:
            global_path = os.path.join(self.evaluation_folder, 'global_feature_importance.png')
            self.plot_global_feature_importance(shap_values, X_test, global_path)

            instance_idx = np.random.randint(0, X_test.shape[0])
            local_path = os.path.join(self.evaluation_folder, f'local_feature_importance_{instance_idx}.png')
            self.plot_local_feature_importance(shap_values, X_test, instance_idx, local_path)


if __name__ == "__main__":
    workflow = MLWorkflow(
        data_folder="data\\Model",
        descriptors_file="DP_Training.csv",
        model_save_path="best_model_overall_best.joblib",
        feature_names_path="feature_names_descriptors.joblib",
        evaluation_folder="Evaluation",
        target_column='RI'
    )
    workflow.main()


Model loaded successfully.
Feature names loaded successfully.
Validation Metrics: {'RMSE': 183.7062936269039, 'MedAE': 96.39153217966702, 'MAE': 130.53660335048517, 'MRE': 0.06698110419193777, 'MedRE': 0.049460542824234614}
Model retrained and saved successfully.
Model Metrics: {'RMSE': 166.64177318066126, 'MedAE': 95.8505650510001, 'MAE': 122.00070529414909, 'MRE': 0.06601769766999409, 'MedRE': 0.05186718888041131}
SHAP values calculated successfully.
Global feature importance plot saved to data\AutoML\Evaluation\global_feature_importance.png
Local feature importance plot for instance 158 saved to data\AutoML\Evaluation\local_feature_importance_158.png


In [ ]:
##########################################
#  Transfer learning based on best model #
##########################################
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, median_absolute_error, mean_absolute_error
from math import sqrt
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
from xgboost import XGBRegressor

# Clean feature names to ensure compatibility with XGBoost
def clean_feature_names(feature_names):
    # Use regular expressions to remove characters that are not letters, numbers, or underscores
    cleaned_feature_names = [re.sub(r'[^a-zA-Z0-9_]', '', str(f)) for f in feature_names]
    return cleaned_feature_names

# Load and clean feature names
def load_feature_names(feature_names_path):
    feature_names = joblib.load(feature_names_path)
    return clean_feature_names(feature_names)

# Process new data for transfer learning
def process_new_data(data_file, feature_names):
    df = pd.read_csv(data_file, low_memory=False)
    if 'ID' in df.columns:
        df = df.drop(columns=['ID'])  # Remove the ID column if present

    # Filter to only include valid feature names
    feature_names = [f for f in feature_names if f in df.columns]
    
    X = df[feature_names].apply(pd.to_numeric, errors='coerce').fillna(0)  # Ensure numerical data and replace NaN with 0
    y = df['RI'].apply(pd.to_numeric, errors='coerce').fillna(0)  # Ensure numerical target variable

    return X, y

# Evaluate the model's performance
def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)
    rmse = sqrt(mean_squared_error(y_test, predictions))
    medae = median_absolute_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    mre = mean_absolute_error(y_test, predictions) / y_test.abs().mean()  # Mean relative error
    medre = median_absolute_error(y_test, predictions) / y_test.abs().mean()  # Median relative error
    
    metrics = {
        "RMSE": rmse,
        "MedAE": medae,
        "MAE": mae,
        "MRE": mre,
        "MedRE": medre
    }
    return metrics, predictions

# Save evaluation results (metrics and predictions)
def save_evaluation_results(metrics, predictions, y_test, output_folder, condition):
    metrics_df = pd.DataFrame([metrics])
    metrics_df.to_csv(os.path.join(output_folder, f"{condition}_metrics.csv"), index=False)
    
    predictions_df = pd.DataFrame({
        'Actual RI': y_test,
        'Predicted RI': predictions
    })
    predictions_df.to_csv(os.path.join(output_folder, f"{condition}_predictions.csv"), index=False)

# Visualize results with scatter plots
def visualize_results(y_test, predictions, output_folder, condition):
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=y_test, y=predictions, alpha=0.7)
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], linestyle='--', color='red')
    plt.title(f"Actual vs. Predicted RI - {condition}")
    plt.xlabel("Actual RI")
    plt.ylabel("Predicted RI")
    plt.savefig(os.path.join(output_folder, f"{condition}_prediction_vs_actual.png"))
    plt.close()

# Optimize the model using GridSearchCV
def optimize_model(X_train, y_train):
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [4, 6, 8],
        'learning_rate': [0.01, 0.05, 0.1],
        'reg_lambda': [1, 3, 5, 7, 9]
    }
    
    model = XGBRegressor()
    grid_search = GridSearchCV(model, param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    
    print(f"Best Parameters: {grid_search.best_params_}")
    print(f"Best CV Score: {grid_search.best_score_}")
    
    return grid_search.best_estimator_

# Save the best model and feature names
def save_best_model_and_features(model, feature_names, model_path, feature_names_path):
    joblib.dump(model, model_path)
    joblib.dump(feature_names, feature_names_path)
    print("Model and feature names saved for future use.")

# Main program for transfer learning
def main():
    feature_names_path = "data/Model/feature_names_descriptors.joblib"
    feature_names = load_feature_names(feature_names_path)

    transform_folder = "data/Model/Transform_learning/"
    conditions = ["DP_TL.csv"]  # List of datasets for transfer learning

    for condition in conditions:
        data_file = os.path.join(transform_folder, condition)
        X, y = process_new_data(data_file, feature_names)
        
        # Split dataset into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
        
        # Perform hyperparameter optimization and transfer learning
        best_model = optimize_model(X_train, y_train)
        
        # Save the best model and feature names
        save_best_model_and_features(best_model, feature_names, os.path.join(transform_folder, "best_transfer_model.joblib"), feature_names_path)
        
        # Evaluate the optimized model
        metrics, predictions = evaluate_model(best_model, X_test, y_test)
        
        # Save evaluation results (metrics and predictions)
        save_evaluation_results(metrics, predictions, y_test, transform_folder, condition)
        
        # Visualize the evaluation results
        visualize_results(y_test, predictions, transform_folder, condition)

if __name__ == "__main__":
    main()


In [ ]:
################################################
#  Unknown data prediction based on best model #
################################################
import pandas as pd
from sklearn.model_selection import train_test_split
from supervised.automl import AutoML
from sklearn.metrics import mean_squared_error, median_absolute_error, mean_absolute_error, roc_auc_score
from math import sqrt
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import multiprocessing
import os

class BestModel:
    def __init__(self, algorithm, metrics, model, predictions, y_test):
        self.algorithm = algorithm
        self.metrics = metrics
        self.model = model
        self.predictions = predictions
        self.y_test = y_test

def core():
    # Returns the minimum between 5 and the number of available CPU cores
    return min(5, multiprocessing.cpu_count())

def calculate_metrics(y_true, y_pred):
    # Calculate evaluation metrics
    rmse = sqrt(mean_squared_error(y_true, y_pred))
    medae = median_absolute_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mre = mean_absolute_error(y_true, y_pred) / max(abs(y_true))
    medre = median_absolute_error(y_true, y_pred) / max(abs(y_true))
    try:
        roc = roc_auc_score(y_true, y_pred)
    except:
        roc = None  # ROC AUC is only applicable for classification tasks
    
    metrics = {
        "RMSE": rmse,
        "MedAE": medae,
        "MAE": mae,
        "MRE": mre,
        "MedRE": medre,
        "ROC": roc
    }
    return metrics

def train_and_evaluate(algorithm, X_train, y_train, X_test, y_test, results_path):
    # Train and evaluate a model using AutoML
    try:
        automl = AutoML(algorithms=[algorithm],
                        mode="Perform",
                        results_path=results_path,
                        eval_metric="rmse",
                        n_jobs=core(),
                        total_time_limit=3600,
                        model_time_limit=3600,
                        train_ensemble=True)
        automl.fit(X_train, y_train)

        pred_test = automl.predict(X_test)
        metrics = calculate_metrics(y_test, pred_test)
        return BestModel(algorithm, metrics, automl, pred_test, y_test)
    except Exception as e:
        print(f"Error occurred with {algorithm} for {results_path}: {e}")
        return None

def process_data(data_file):
    # Process data by reading the CSV file and converting all columns to numeric
    df = pd.read_csv(data_file, low_memory=False)
    X = df.loc[:, df.columns != 'RI'].apply(pd.to_numeric, errors='coerce')
    y = df['RI'].apply(pd.to_numeric, errors='coerce')
    return X, y

def save_metrics_to_csv(best_model_info, data_folder, feature_type):
    # Save evaluation metrics to a CSV file
    metrics_df = pd.DataFrame([best_model_info.metrics])
    metrics_df.insert(0, 'Algorithm', best_model_info.algorithm)
    metrics_df.to_csv(os.path.join(data_folder, f"{feature_type}_model_metrics.csv"), index=False)

def save_predictions_to_csv(best_model_info, data_folder, feature_type):
    # Save actual and predicted values to a CSV file
    predictions_df = pd.DataFrame({
        'Actual RI': best_model_info.y_test,
        'Predicted RI': best_model_info.predictions
    })
    predictions_df.to_csv(os.path.join(data_folder, f"{feature_type}_predictions.csv"), index=False)

def predict_unknown_data(model_file, unknown_data_file, output_file, feature_names_file):
    # Load the model and feature names
    model = joblib.load(model_file)
    feature_names = joblib.load(feature_names_file)
    unknown_data = pd.read_csv(unknown_data_file, low_memory=False)
    
    # Extract the ID column
    ids = unknown_data['ID'] if 'ID' in unknown_data.columns else None
    
    # Fill missing feature columns with 0
    for feature in feature_names:
        if feature not in unknown_data.columns:
            unknown_data[feature] = 0

    # Retain the same feature order as during training
    unknown_data = unknown_data[feature_names]
    
    # Convert all columns to numeric, set unconvertible values to NaN, and fill NaN with 0
    unknown_data = unknown_data.apply(pd.to_numeric, errors='coerce').fillna(0)
    
    # Check if the dataset is empty after processing
    if unknown_data.empty:
        print("Error: After processing, the unknown data is empty. Please check the input file and processing steps.")
        return
    
    # Perform predictions
    predictions = model.predict(unknown_data)
    
    # Combine IDs and predictions
    predictions_df = pd.DataFrame({'ID': ids, 'Predicted RI': predictions}) if ids is not None else pd.DataFrame({'Predicted RI': predictions})
    predictions_df.to_csv(output_file, index=False)
    print(f"Predictions saved to {output_file}")

def main():
    data_folder = "data/Model"
    descriptors_file = os.path.join(data_folder, "DP_Training.csv")
    fingerprints_file = os.path.join(data_folder, "FP_Training.csv")
    model_save_path = os.path.join(data_folder, "best_model")

    descriptors_folder = os.path.join(data_folder, "DP_model")
    fingerprints_folder = os.path.join(data_folder, "FP_model")
    os.makedirs(descriptors_folder, exist_ok=True)
    os.makedirs(fingerprints_folder, exist_ok=True)

    # Process datasets
    X_descriptors, y_descriptors = process_data(descriptors_file)
    X_fingerprints, y_fingerprints = process_data(fingerprints_file)

    # Split datasets into training and testing sets
    X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_descriptors, y_descriptors, test_size=0.20, random_state=42)
    X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(X_fingerprints, y_fingerprints, test_size=0.20, random_state=42)

    all_algorithms = ["Xgboost", "CatBoost", "Decision Tree", "Extra Trees", "LightGBM"]

    # Train models for descriptors
    results_d = joblib.Parallel(n_jobs=core())(
        joblib.delayed(train_and_evaluate)(algorithm, X_train_d, y_train_d, X_test_d, y_test_d, os.path.join(descriptors_folder, f"AutoML_{algorithm}"))
        for algorithm in all_algorithms
    )

    # Train models for fingerprints
    results_f = joblib.Parallel(n_jobs=core())(
        joblib.delayed(train_and_evaluate)(algorithm, X_train_f, y_train_f, X_test_f, y_test_f, os.path.join(fingerprints_folder, f"AutoML_{algorithm}"))
        for algorithm in all_algorithms
    )

    # Identify the best models
    best_model_info_d = None
    best_rmse_d = float('inf')
    best_model_info_f = None
    best_rmse_f = float('inf')

    for result in results_d:
        if result and result.metrics["RMSE"] < best_rmse_d:
            best_rmse_d = result.metrics["RMSE"]
            best_model_info_d = result

    for result in results_f:
        if result and result.metrics["RMSE"] < best_rmse_f:
            best_rmse_f = result.metrics["RMSE"]
            best_model_info_f = result

    # Save the best models and metrics
    if best_model_info_d:
        joblib.dump(best_model_info_d.model, f"{model_save_path}_descriptors.joblib")
        joblib.dump(X_descriptors.columns, os.path.join(descriptors_folder, "feature_names_descriptors.joblib"))
        save_metrics_to_csv(best_model_info_d, descriptors_folder, "descriptors")
        save_predictions_to_csv(best_model_info_d, descriptors_folder, "descriptors")

    if best_model_info_f:
        joblib.dump(best_model_info_f.model, f"{model_save_path}_fingerprints.joblib")
        joblib.dump(X_fingerprints.columns, os.path.join(fingerprints_folder, "feature_names_fingerprints.joblib"))
        save_metrics_to_csv(best_model_info_f, fingerprints_folder, "fingerprints")
        save_predictions_to_csv(best_model_info_f, fingerprints_folder, "fingerprints")

    # Determine the overall best model
    if best_model_info_d and best_model_info_f:
        overall_best_model = best_model_info_d if best_model_info_d.metrics["RMSE"] < best_model_info_f.metrics["RMSE"] else best_model_info_f
    elif best_model_info_d:
        overall_best_model = best_model_info_d
    elif best_model_info_f:
        overall_best_model = best_model_info_f
    else:
        overall_best_model = None

    if overall_best_model:
        joblib.dump(overall_best_model.model, f"{model_save_path}_overall_best.joblib")
        feature_names_path = os.path.join(data_folder, "feature_names_descriptors.joblib") if overall_best_model == best_model_info_d else os.path.join(data_folder, "feature_names_fingerprints.joblib")
        joblib.dump(X_descriptors.columns if overall_best_model == best_model_info_d else X_fingerprints.columns, feature_names_path)
        save_predictions_to_csv(overall_best_model, data_folder, "overall_best")

        # Predict unknown data using the best models
        unknown_data_file_d = os.path.join(data_folder, "DP_Training.csv")
        output_predictions_file_d = os.path.join(data_folder, "known_predictions_descriptors.csv")
        predict_unknown_data(f"{model_save_path}_descriptors.joblib", unknown_data_file_d, output_predictions_file_d, os.path.join(descriptors_folder, "feature_names_descriptors.joblib"))

        unknown_data_file_f = os.path.join(data_folder, "FP_Training.csv")
        output_predictions_file_f = os.path.join(data_folder, "known_predictions_fingerprints.csv")
        predict_unknown_data(f"{model_save_path}_fingerprints.joblib", unknown_data_file_f, output_predictions_file_f, os.path.join(fingerprints_folder, "feature_names_fingerprints.joblib"))

if __name__ == "__main__":
    main()


In [26]:
#############################################################
#  Unknown data prediction based on Transfer Learning model #
#############################################################
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, median_absolute_error, mean_absolute_error
from math import sqrt
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
from xgboost import XGBRegressor

# Clean feature names to ensure compatibility with XGBoost
def clean_feature_names(feature_names):
    cleaned_feature_names = [re.sub(r'[^a-zA-Z0-9_]', '', str(f)) for f in feature_names]
    return cleaned_feature_names

# Load and clean feature names
def load_feature_names(feature_names_path):
    feature_names = joblib.load(feature_names_path)
    return clean_feature_names(feature_names)

# Process new data for training or prediction
def process_new_data(data_file, feature_names):
    df = pd.read_csv(data_file, low_memory=False)
    if 'ID' in df.columns:
        ids = df['ID']
        df = df.drop(columns=['ID'])
    else:
        ids = None
    
    # Filter and retain only valid features
    feature_names = [f for f in feature_names if f in df.columns]
    X = df[feature_names].apply(pd.to_numeric, errors='coerce').fillna(0)
    y = df['RI'].apply(pd.to_numeric, errors='coerce').fillna(0) if 'RI' in df.columns else None

    return X, y, ids

# Evaluate model performance
def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)
    rmse = sqrt(mean_squared_error(y_test, predictions))
    medae = median_absolute_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    mre = mae / y_test.abs().mean()
    medre = medae / y_test.abs().mean()
    
    metrics = {
        "RMSE": rmse,
        "MedAE": medae,
        "MAE": mae,
        "MRE": mre,
        "MedRE": medre
    }
    return metrics, predictions

# Save evaluation results
def save_evaluation_results(metrics, predictions, y_test, ids, output_folder, condition):
    metrics_df = pd.DataFrame([metrics])
    metrics_df.to_csv(os.path.join(output_folder, f"{condition}_metrics.csv"), index=False)
    
    # Include ID column in the output if available
    if ids is not None and len(ids) == len(y_test):
        predictions_df = pd.DataFrame({
            'ID': ids,
            'Actual RI': y_test,
            'Predicted RI': predictions
        })
    else:
        predictions_df = pd.DataFrame({
            'Actual RI': y_test,
            'Predicted RI': predictions
        })
    
    predictions_df.to_csv(os.path.join(output_folder, f"{condition}_predictions.csv"), index=False)

# Visualize actual vs. predicted results
def visualize_results(y_test, predictions, output_folder, condition):
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=y_test, y=predictions, alpha=0.7)
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], linestyle='--', color='red')
    plt.title(f"Actual vs. Predicted RI - {condition}")
    plt.xlabel("Actual RI")
    plt.ylabel("Predicted RI")
    plt.savefig(os.path.join(output_folder, f"{condition}_prediction_vs_actual.png"))
    plt.close()

# Optimize the model using GridSearchCV
def optimize_model(X_train, y_train):
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [4, 6, 8],
        'learning_rate': [0.01, 0.05, 0.1],
        'reg_lambda': [1, 3, 5, 7, 9]
    }
    
    model = XGBRegressor()
    grid_search = GridSearchCV(model, param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    
    print(f"Best Parameters: {grid_search.best_params_}")
    print(f"Best CV Score: {grid_search.best_score_}")
    
    return grid_search.best_estimator_

# Save the best model and feature names
def save_best_model_and_features(model, feature_names, model_path, feature_names_path):
    joblib.dump(model, model_path)
    joblib.dump(feature_names, feature_names_path)
    print("Model and feature names saved for future use.")

# Predict unknown data
def predict_unknown_data(model, unknown_data_file, output_file, feature_names):
    X_unknown, _, ids = process_new_data(unknown_data_file, feature_names)
    
    # Perform predictions
    predictions = model.predict(X_unknown)
    
    # Save predictions to a CSV file
    predictions_df = pd.DataFrame({'ID': ids, 'Predicted RI': predictions})
    predictions_df.to_csv(output_file, index=False)
    print(f"Predictions for unknown data saved to {output_file}")

# Main program for transfer learning and prediction
def main():
    feature_names_path = "data/Model/feature_names_descriptors.joblib"
    feature_names = load_feature_names(feature_names_path)

    transform_folder = "data/Model/Transform_learning/"
    conditions = ["DP_TL.csv"]

    # Transfer learning and evaluation
    for condition in conditions:
        data_file = os.path.join(transform_folder, condition)
        X, y, ids = process_new_data(data_file, feature_names)
        
        # Split the dataset into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
        
        # Hyperparameter optimization and transfer learning
        best_model = optimize_model(X_train, y_train)
        
        # Save the best model and feature names
        save_best_model_and_features(best_model, feature_names, os.path.join(transform_folder, "best_transfer_model.joblib"), feature_names_path)
        
        # Evaluate the optimized model
        metrics, predictions = evaluate_model(best_model, X_test, y_test)
        
        # Save evaluation results
        save_evaluation_results(metrics, predictions, y_test, ids, transform_folder, condition)
        
        # Visualize the results
        visualize_results(y_test, predictions, transform_folder, condition)

    # Load the best model and predict unknown data
    best_model = joblib.load(os.path.join(transform_folder, "best_transfer_model.joblib"))
    unknown_data_file = "data/Model/merged_DP_data.csv"
    output_predictions_file = os.path.join(transform_folder, "unknown_predictions.csv")
    
    # Predict unknown data
    predict_unknown_data(best_model, unknown_data_file, output_predictions_file, feature_names)

if __name__ == "__main__":
    main()


Fitting 3 folds for each of 135 candidates, totalling 405 fits
Best Parameters: {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 300, 'reg_lambda': 9}
Best CV Score: -25718.496145655965
Model and feature names saved for future use.
Predictions for unknown data saved to data/AutoML/Transform_learning/unknown_predictions.csv
